In [290]:
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors


# Trabajar con DataFrames
# -----------------------------------------------------------------------
import pandas as pd

## Conexión

In [287]:
try:
    conexion = psycopg2.connect(
        database = "Ubicaciones",
        user = "my_user",
        password = "admin",
        host = "localhost",
        port = "5432"
    )
except OperationalError as e:
    if e.pgcode == errorcodes.INVALID_PASSWORD:
        print("Contraseña es errónea")
    elif e.pgcode == errorcodes.CONNECTION_EXCEPTION:
        print("Error de conexión")
    else:
        print(f"Ocurrió el error {e}") 

cursor = conexion.cursor()

## Tratamiento de datos

In [91]:
df_fsq = pd.read_csv("datos/api_foursquare.csv", index_col = 0)
df_municipios = pd.read_csv("datos/municipios.csv", index_col = 0)
df_aemet = pd.read_csv("datos/df_aemet_final.csv", index_col = 0)

In [255]:
df_aemet = pd.read_csv("datos/df_aemet_final.csv", index_col = 0)

In [256]:
df_aemet.reset_index(inplace=True)
df_aemet.head(1)

,fecha,cielo,temp._(°c),sen._térmica_(°c),racha_máx._(km/h),precipitación_(mm),nieve_(mm),humedad_relativa_(%),prob._precip.__(%),prob._de_nieve_(%),prob._de_tormenta_(%),avisos,dirección_viento,velocidad_del_viento,localizacion_id
0,2024-08-29 12:00:00,Nubes altas,21,21,25.0,0.0,0,69,95,0,75,Riesgo,S,7,acebeda-la-id28001


In [257]:
df_aemet[['localizacion', 'id']] = df_aemet['localizacion_id'].str.split('-id', expand=True)
df_aemet[['fecha', 'hora']] = df_aemet['fecha'].str.split(' ', expand=True)

In [258]:
df_aemet = df_aemet.drop(columns = "localizacion_id") #porque ya la había separado
df_aemet = df_aemet.drop(columns = "avisos") #porque no me aporta valor, no sé porqué es el riesgo
df_aemet = df_aemet.drop(columns = "prob._precip.__(%)") #me quedo con los mm precipitación, porque hay probabilidades del 100% con mm 0.
df_aemet = df_aemet.drop(columns = "id") #porque es un id que no puedo relacionar con otras tablas o me aporta info adicional
df_aemet.rename(columns={'localizacion': 'municipio_id'}, inplace=True)
df_aemet.head(1)

,fecha,cielo,temp._(°c),sen._térmica_(°c),racha_máx._(km/h),precipitación_(mm),nieve_(mm),humedad_relativa_(%),prob._de_nieve_(%),prob._de_tormenta_(%),dirección_viento,velocidad_del_viento,municipio_id,hora
0,2024-08-29,Nubes altas,21,21,25.0,0.0,0,69,0,75,S,7,acebeda-la,12:00:00


In [ ]:
df_fsq.rename(columns={'municipio': 'municipio_id'}, inplace=True)
df_fsq = df_aemet.drop(columns = "fsq_id") #lo quito porque ya está contenido detrás de la tercera barra del link

In [193]:
df_fsq['latitude'] = df_fsq['latitude'].astype(float)
df_fsq['longitude'] = df_fsq['longitude'].astype(float)

In [194]:
df_fsq.head(1)

,municipio_id,category,closed_bucket,distance,link,name,address,latitude,longitude
0,ajalvir,Park,LikelyOpen,563.0,/v3/places/4d3c6c5b84d46ea87dddfd5c,Diverjungla,"Calle Segovia, 20 (Pol. Ind. Donada), 28864 Aj...",40.529921,-3.484341


In [70]:
df_municipios = df_municipios.T
df_municipios.reset_index(inplace=True)
df_municipios

,index,0,1,2
0,acebeda-la,41.0869583,-3.624399,La Acebeda
1,ajalvir,40.5342302,-3.4807818,Ajalvir
2,alameda-del-valle,40.9177178,-3.8438216,Alameda del Valle
3,alamo-el,40.2307296,-3.9905893,El Álamo
4,alcala-de-henares,40.4818396,-3.3644973,Alcalá de Henares
...,...,...,...,...
171,villar-del-olmo,40.3364465,-3.2355624,Villar del Olmo
172,villarejo-de-salvanes,40.1683582,-3.2738764,Villarejo de Salvanés
173,villaviciosa-de-odon,40.3573787,-3.9002334,Villaviciosa de Odón
174,villavieja-del-lozoya,41.0063622,-3.6716291,Villavieja del Lozoya


In [76]:
df_municipios.rename(columns={'index': 'municipio_id', 0: 'latitud municipio', 1:'longitud municipio',2:'nombre limpio municipio'}, inplace=True)

In [108]:
df_municipios

,municipio_id,latitud municipio,longitud municipio,nombre limpio municipio
0,acebeda-la,41.0869583,-3.624399,La Acebeda
1,ajalvir,40.5342302,-3.4807818,Ajalvir
2,alameda-del-valle,40.9177178,-3.8438216,Alameda del Valle
3,alamo-el,40.2307296,-3.9905893,El Álamo
4,alcala-de-henares,40.4818396,-3.3644973,Alcalá de Henares
...,...,...,...,...
171,villar-del-olmo,40.3364465,-3.2355624,Villar del Olmo
172,villarejo-de-salvanes,40.1683582,-3.2738764,Villarejo de Salvanés
173,villaviciosa-de-odon,40.3573787,-3.9002334,Villaviciosa de Odón
174,villavieja-del-lozoya,41.0063622,-3.6716291,Villavieja del Lozoya


In [188]:
df_municipios['latitud municipio'] = df_municipios['latitud municipio'].astype(float)
df_municipios['longitud municipio'] = df_municipios['longitud municipio'].astype(float)

In [189]:
df_municipios

,municipio_id,latitud municipio,longitud municipio,nombre limpio municipio
0,acebeda-la,-3.624399,-3.624399,La Acebeda
1,ajalvir,-3.480782,-3.480782,Ajalvir
2,alameda-del-valle,-3.843822,-3.843822,Alameda del Valle
3,alamo-el,-3.990589,-3.990589,El Álamo
4,alcala-de-henares,-3.364497,-3.364497,Alcalá de Henares
...,...,...,...,...
171,villar-del-olmo,-3.235562,-3.235562,Villar del Olmo
172,villarejo-de-salvanes,-3.273876,-3.273876,Villarejo de Salvanés
173,villaviciosa-de-odon,-3.900233,-3.900233,Villaviciosa de Odón
174,villavieja-del-lozoya,-3.671629,-3.671629,Villavieja del Lozoya


In [103]:
print(df_aemet['localizacion'].nunique())
print(df_fsq['municipio'].nunique())
print(df_municipios['municipio fsq'].nunique())

179
96
176


In [234]:
set_aemet = set(df_aemet['municipio_id'])
set_municipios = set(df_municipios['municipio_id'])

In [235]:
diferencia = set_aemet-set_municipios

In [237]:
diferencia

{'cabrera-la', 'leganes', 'molinos-los'}

In [272]:
df_aemet = df_aemet.loc[df_aemet['municipio_id'] != 'cabrera-la']
df_aemet = df_aemet.loc[df_aemet['municipio_id'] != 'leganes']
df_aemet = df_aemet.loc[df_aemet['municipio_id'] != 'molinos-los']

In [273]:
print(df_aemet['municipio_id'].nunique())
print(df_fsq['municipio_id'].nunique())
print(df_municipios['municipio_id'].nunique())

176
96
176


In [128]:
nuevo_df_cielo = df_aemet[['municipio_id', 'fecha', 'cielo']]

In [129]:
nuevo_df_cielo

,municipio_id,fecha,cielo
0,acebeda-la,2024-08-29 12:00:00,Nubes altas
1,acebeda-la,2024-08-29 13:00:00,Nubes altas
2,acebeda-la,2024-08-29 14:00:00,Muy nuboso
3,acebeda-la,2024-08-29 15:00:00,Nubes altas
4,acebeda-la,2024-08-29 16:00:00,Nubes altas
...,...,...,...
7880,zarzalejo,2024-08-31 03:00:00,Poco nuboso
7881,zarzalejo,2024-08-31 04:00:00,Cielo despejado
7882,zarzalejo,2024-08-31 05:00:00,Cielo despejado
7883,zarzalejo,2024-08-31 06:00:00,Poco nuboso


In [115]:
df_aemet['cielo'].value_counts()

cielo
Poco nuboso                                1442
Cubierto                                   1317
Nubes altas                                1210
Muy nuboso                                  929
Cielo despejado                             862
Intervalos nubosos con lluvia escasa        733
Nuboso                                      529
Cubierto con lluvia escasa                  230
Muy nuboso con lluvia escasa                135
Cubierto con tormenta y lluvia escasa       126
Intervalos nubosos                           89
Cubierto con lluvia                          88
Nuboso con lluvia escasa                     67
Muy nuboso con tormenta y lluvia escasa      44
Nuboso con tormenta y lluvia escasa          22
Intervalos nubosos con lluvia                19
Cubierto con tormenta                        12
Bruma                                        10
Muy nuboso con lluvia                         9
Niebla                                        7
Muy nuboso con tormenta           

In [121]:
df_fsq['category'].value_counts()

category
Park                                                738
Monument                                            126
TV Station                                           24
Bridge                                                6
Castle                                                3
Film Studio                                           3
Fountain                                              3
Beach                                                 2
Palace                                                1
Print, TV, Radio and Outdoor Advertising Service      1
Name: count, dtype: int64

In [112]:
df_fsq['category'].unique()

array(['Park', 'Monument', 'TV Station', 'Castle', 'Film Studio',
       'Palace', 'Fountain', 'Bridge', 'Beach',
       'Print, TV, Radio and Outdoor Advertising Service'], dtype=object)

In [274]:
lista_tuplas_municipios = []
for fila in df_municipios.values:
    lista_tuplas_municipios.append(tuple(fila))

lista_tuplas_fsq = []
for fila in df_fsq.values:
    lista_tuplas_fsq.append(tuple(fila))

lista_tuplas_aemet = []
for fila in df_aemet.values:
    lista_tuplas_aemet.append(tuple(fila))


## Creación de tablas

In [145]:
query_creacion_municipios = '''
    create table if not exists municipios(
        id_municipio VARCHAR(100) primary key, 
        latitud INT,
        longitud INT,
        nombre VARCHAR(100)
        ); '''

cursor.execute(query_creacion_municipios)
conexion.commit()

In [153]:
query_creacion_locales = '''
    create table if not exists locales (
        id_local SERIAL primary key,
        id_municipio VARCHAR(100)
            references municipios(id_municipio)
            on update cascade,
        categoria CHAR(50),
        estado CHAR(50),
        distancia INT,
        link VARCHAR(100),
        nombre VARCHAR(100),
        direccion VARCHAR(300),
        latitud DECIMAL,
        longitud DECIMAL
    );'''

cursor.execute(query_creacion_locales)
conexion.commit()

In [288]:
query_creacion_clima = '''
    create table if not exists clima (
        id_clima SERIAL primary key,
        fecha DATE,
        cielo VARCHAR(50),
        temperatura DECIMAL,
        sensacion DECIMAL,
        racha_max DECIMAL,
        precipitacion DECIMAL,
        nieve DECIMAL,
        humedad_relativa DECIMAL,
        prob_nieve DECIMAL,
        prob_tormenta DECIMAL,
        direccion_viento VARCHAR(10),
        velocidad_viento DECIMAL,
        municipio_id VARCHAR(100)
            references municipios(id_municipio)
            on update cascade,
        hora VARCHAR (20)
    );'''

cursor.execute(query_creacion_clima)
conexion.commit()

In [192]:
query_insercion_municipios = '''
    insert into municipios (id_municipio, latitud, longitud, nombre)
    values
    (%s, %s, %s, %s)''' 

cursor.executemany(query_insercion_municipios, lista_tuplas_municipios) 
conexion.commit()

In [196]:
query_insercion_locales = '''
    insert into locales (id_municipio, categoria, estado, distancia, link, nombre, direccion, latitud, longitud)
    values
    (%s, %s, %s, %s, %s, %s, %s, %s, %s)''' 

cursor.executemany(query_insercion_locales, lista_tuplas_fsq) 
conexion.commit()

In [289]:
query_insercion_clima = '''
    insert into clima (fecha, cielo, temperatura, sensacion, racha_max, precipitacion, nieve, humedad_relativa, prob_nieve, prob_tormenta, direccion_viento, velocidad_viento, municipio_id, hora)
    values
    (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'''

cursor.executemany(query_insercion_clima, lista_tuplas_aemet)
conexion.commit()

## Queries

#### 1. Contar municipios con más de 5 locales

In [321]:
query1 = '''
    select m.nombre, count(l.id_local)
    from municipios m
    inner join locales l on
        m.id_municipio = l.id_municipio
        
    group by m.nombre
    having count(l.id_local) > 5;
'''

cursor.execute(query1)
df_municipios_locales = pd.DataFrame(cursor.fetchall())

In [323]:
df_municipios_locales.head(5)

,0,1
0,Collado Villalba,13
1,Real Monasterio de San Lorenzo de El Escorial,16
2,Coslada,13
3,Villanueva del Pardillo,6
4,Móstoles,37


#### 2. Municipio con la temperatura más alta

In [305]:
query2 = '''
    select m.nombre, c.temperatura 
    from clima c
    inner join municipios m
        on c.municipio_id = m.id_municipio
    order by c.temperatura desc
    limit 1;
    '''

cursor.execute(query2)
df_temp_mas_alta = pd.DataFrame(cursor.fetchall()).head() 

In [306]:
df_temp_mas_alta

,0,1
0,Aldea del Fresno,32


In [313]:
lista_tuplas_aemet[0][0]

'2024-08-29'

In [343]:
lista_tuplas_aemet[5][13]

'17:00:00'

#### 3. Locales a menos de 5km del centro del municipiop con cielo nublado el 30 de agosto a las 17hrs


#### 4. Locales en municipios con cielo despejado el 29 de agosto de 2024

In [325]:
query4 = '''
    select l.nombre
    from locales l
    inner join clima c on l.id_municipio = c.municipio_id
    where c.cielo = 'Cielo despejado' and c.fecha = '2024-08-29';'''

cursor.execute(query4)
df_despejado = pd.DataFrame(cursor.fetchall())

In [326]:
df_despejado

,0
0,Digi24
1,Monumento a los Reyes Católicos
2,Parroquia de Nuestra Señora de la Moraleja
3,Parque de la Marina
4,Parque de las Cepas
...,...
292,Parque Enrique Tierno Galván
293,Los Lagos
294,Parque Forestal Cerro del Castillo
295,Parque Infantil


#### 5. Temperatura promedio por municipio

In [344]:
query5 = '''
    select m.nombre, round(avg(c.temperatura), 2)
    from clima c
    join municipios m on c.municipio_id = m.id_municipio
    group by m.nombre;
'''

cursor.execute(query5)
df_temp_promedio = pd.DataFrame(cursor.fetchall())

In [345]:
df_temp_promedio.head(5)

,0,1
0,Villavieja del Lozoya,19.32
1,Redueña,20.91
2,Camarma de Esteruelas,22.11
3,Collado Villalba,21.73
4,Villamantilla,23.20
